In [1]:
%cd ..
import numpy as np
from skeletrack.io import read_skeleton_file
body_info = read_skeleton_file('dataset/raw/clapping/S001C001P001R001A010.skeleton')

/beegfs/xg703/Vision2018-RL


# Preprocessing

In [2]:
from skeletrack.preprocessing import preprocess
preprocess(body_info, gamma=0.95, is_clapping=True)

In [3]:
body_info[0][0].joints[
    ['v1','v2','v3','d1','d2','d3','a1','a2','a3']
].head()

,v1,v2,v3,d1,d2,d3,a1,a2,a3
0,1.008033,-0.122525,-0.059198,0.0,0.0,0.0,-0.000023,-0.000325,0.000022
1,0.501079,-0.065888,-0.041108,0.0,0.0,0.0,-0.000002,-0.000177,0.000039
2,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000
3,-0.265429,-0.065333,0.018349,0.0,0.0,0.0,0.000179,0.000143,-0.000244
4,0.182357,-0.319248,-0.037038,0.0,0.0,0.0,0.000680,-0.000056,-0.000322


In [4]:
body_info[0][0].weight

0.02903546361765786

## Sanity checks

### 1. Coordinate transformation: special joints

In [5]:
for i in range(len(body_info)):
    for j in range(len(body_info[0])):
        joints = body_info[i][j].joints
        neck = joints.loc[2][['v1','v2','v3']]
        right_hip = joints.loc[16][['v1','v2','v3']]
        assert np.all(np.isclose(neck, np.zeros(3)))
        assert np.all(np.isclose(right_hip, np.array([1., 0., 0.])))

### 2. Displacements

In [6]:
for i in range(1, len(body_info)):
    for j in range(len(body_info[0])):
        vprev = body_info[i-1][j].joints[['v1', 'v2', 'v3']]
        vthis = body_info[i][j].joints[['v1', 'v2', 'v3']]
        dthis = body_info[i][j].joints[['d1', 'd2', 'd3']]
        assert np.all(np.isclose(vthis - vprev, dthis))

### 3. Actions

In [7]:
for i in range(1, len(body_info) - 1):
    for j in range(len(body_info[0])):
        vprev = body_info[i-1][j].joints[['v1', 'v2', 'v3']]
        vthis = body_info[i][j].joints[['v1', 'v2', 'v3']]
        vnext = body_info[i+1][j].joints[['v1', 'v2', 'v3']]
        athis = body_info[i][j].joints[['a1', 'a2', 'a3']]
        assert np.all(np.isclose(vnext + vprev - 2*vthis, athis))